In [66]:
import ollama

prompt = "เมืองหลวงของประเทศไทย ?"

response = ollama.chat(
    model="llama3.2",
    messages=[
        {
          "role" : "system",
          "content" : "You are female assistant helpful with Answer with นะคะ"  
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
)

response["message"]["content"]


'กรุงเทพฯ คือ เมืองหลวงของประเทศไทยนะคะ'

In [67]:
import psycopg2

def get_db_connection():
    conn = psycopg2.connect(
        host="localhost",
        database="mydatabase",
        user="myuser",
        password="mypassword",
        port=5432,
    )
    return conn

def get_db_cursor(conn):
    return conn.cursor()

conn = get_db_connection()
cur = get_db_cursor(conn)
# if not exists wil create extension vector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;") # new type vector

#  create table save vector embedding
cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    content TEXT,
    embedding VECTOR(1024) -- Adjust the dimension based on your model
);
""")

conn.commit()
cur.close()
conn.close()


In [68]:
from sentence_transformers import SentenceTransformer

# initial conn instance
conn = get_db_connection()
cur = get_db_cursor(conn)
embedder = SentenceTransformer("BAAI/bge-m3")

def add_document(text):
    text_trim = text.replace("\n", " ")
    embedding = embedder.encode(text_trim, show_progress_bar=True).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()


documents = [
    "สาขาวิชาเทคโนโลยีสารสนเทศ Master of Science Programme in Information Technology หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาเทคโนโลยีสารสนเทศ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาการและ/หรือวิชาชีพ",
    "หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิศวกรรมซอฟต์แวร์เพื่อปัญญาประดิษฐ์ Master of Science Programme in Software Engineering for for Artificial Intelligence หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาวิศวกรรมซอฟต์แวร์เพื่อปัญญาประดิษฐ์ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาการและ/หรือวิชาชีพ ",
    "หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ Master of Science Programme in Intelligent Business Information System หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาชีพ ",
    "หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิทยาการคอมพิวเตอร์ Master of Science Program in Computer Science หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาการและ/หรือวิชาชีพ ",
    "คณะเทคโนโลยีสารสนเทศ มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี เริ่มดำเนินงานมาตั้งแต่ปีงบประมาณ 2538 และเป็นส่วนราชการตามพระราชกฤษฎีกาแบ่งส่วนราชการสถาบันเทคโนโลยีพระจอมเกล้าธนบุรี ตามราชกิจจานุเบกษา ฉบับที่ 114 วันที่ 21 พฤษภาคม 2540 โดยมี ดร.กฤษณพงศ์ กีรติกร เป็นประธานโครงการจัดตั้งคณะฯ โดยมีวัตถุประสงค์ผลิตบัณฑิตระดับปริญญาตรี และสูงกว่าปริญญาตรี ทางเทคโนโลยีสารสนเทศ เพื่อส่งเสริมการพัฒนาอุตสาหกรรมสารสนเทศ ส่งเสริมการพัฒนาและการวิจัยประยุกต์ทางเทคโนโลยีสารสนเทศ และเพื่อให้บริการวิชาการคอมพิวเตอร์และการประมวลสารสนเทศ แก่หน่วยงานทั้งภายในและภายนอกมหาวิทยาลัยฯ การดำเนินงานในระยะแรกใช้พื้นที่บริเวณชั้น 3 อาคารเรียนรวม 2 ของมหาวิทยาลัยฯ และต่อมาได้จึงได้ขยายมาสร้างอาคารคณะเทคโนโลยีสารสนเทศ เมื่อวันที่ 6 ธันวาคม 2545 จนถึงปัจจุบัน นอกจากนั้นยังมีการจัดตั้งและดำเนินการโครงการวิทยาเขตราชบุรี และดำเนินการเรียนการสอนและผลิตบัณฑิตระหว่างปี 2539 – 2551 โดยมีรูปแบบการเรียนการสอนผ่านทางไกล (Video Conference) โดยอาจารย์สอนสลับที่ทั้งกรุงเทพฯ และราชบุรี โดยมีวัตถุประสงค์เพื่อขยายโอกาสทางการศึกษาสู่ภูมิภาคอีกด้วย",
    "สัญลักษณ์ประจำคณะเทคโนโลยีสารสนเทศ “Information Evolution” เป็นประติมากรรมสัญลักษณ์ประจำคณะเทคโนโลยีสารสนเทศ ออกแบบและสรรค์สร้างโดยอาจารย์ชำเรือง วิเชียรเขตต์ ศิลปินแห่งชาติ สาขาทัศนศิลป์ (ประติมากรรม) โดยมีความหมายว่า “ตราบใดที่โลกยังมีสิ่งมีชีวิต การสื่อสารที่ฉับไวของมวลมนุษย์ชาติจะครอบคลุมไปทั่วโลก” ตัวอักษรสัญลักษณ์ (Wordmark) SIT เป็นตัวอักษรชื่อย่อของคณะเทคโนโลยีสารสนเทศ ย่อมาจาก School of Information Technology ที่ถูกออกแบบมาอย่างสร้างสรรค์สื่อถึงความเป็นคณะเทคโนโลยีสารสนเทศได้อย่างร่วมสมัย การใช้ชื่อย่อภาษาอังกฤษยังสื่อถึงวิสัยทัศน์ของคณะที่จะพัฒนาคณะให้เป็นคณะเทคโนโลยีสารสนเทศชั้นนำของประเทศ ดาวน์โหลดตราสัญลักษณ์ ",
    "บุคลากร ผศ. ดร.ณรงค์ฤทธิ์ วราภรณ์ - คณบดีคณะเทคโนโลยีสารสนเทศ รศ. ดร.วชิรศักดิ์ วานิชชา - รองคณบดีอาวุโสด้านวิจัยและบริการวิชาการ ผศ. ดร.อุมาพร สุภสิทธิเมธี - รองคณบดีด้านบริหาร ผศ. ดร.วรรัตน์ กระทู้ - รองคณบดีด้านวิชาการ ดร.สยาม แย้มแสงสังข์ - รองคณบดีด้านประกันคุณภาพและพัฒนานักศึกษา ผศ. ดร.ชลเมธ อาปณิกานนท์ - รองคณบดีด้านพัฒนาความเป็นสากล"
]

for doc in documents:
    add_document(doc)

cur.close()
conn.close()




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
def query_vector(query_text , k=5):
    query_embedding = embedder.encode(query_text, show_progress_bar=True).tolist()
    
    conn = get_db_connection()
    cur = get_db_cursor(conn)
    query_embedding_str = "[" + ", ".join(map(str, query_embedding)) + "]"
    cur = conn.cursor()
    
    # Use the <=> operator for similarity search
    # ORDER BY ASC if first will near matches
    # ORDER BY DESC if first will far matches
    query_sql = """
        SELECT content, embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s;
    """
    cur.execute(query_sql, (query_embedding_str, k))
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results
    
    
results = query_vector("หลักสูตร ปริญญาตรี มีอะไรบ้าง ?")
results
    

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[('หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิทยาการคอมพิวเตอร์ Master of Science Program in Computer Science หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาการและ/หรือวิชาชีพ ',
  0.39941179161561735),
 ('หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิทยาการคอมพิวเตอร์ Master of Science Program in Computer Science หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาการและ/หรือวิชาชีพ ',
  0.39941179161561735),
 ('หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ Master of Science Programme in Intelligent Business Information System หลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ เป็นหลักสูตร 2 ปี (4 ภาคการศึกษา) จำนวนหน่วยกิตรวม 36 หน่วยกิต เน้นสายวิชาชีพ ',
  0.39942581660439347),
 ('หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ Master of Science Programm

In [70]:
import ollama

def generate_response(query_text):
    retrieved_docs = query_vector(query_text , 3)
    
    context = "\n".join([doc[0] for doc in retrieved_docs])
    
    prompt = f"Answer the question based following context:\n {context} \n\n Question: {query_text} \n\n"
    
    response = ollama.chat(
      model="llama3.2",
      messages=[
          { "role" : "system", "content" : "You are an assistant about 'มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี คณะเทคโนโลยีสารสนเทศ' only and provide data to user , if user's question no data you say to user no data" },
          { "role": "user", "content": prompt }
          
      ]   
    )
    
    return f"{query_text} : {response['message']['content']}"

result = [   
  generate_response("คณบดีคือใคร ?"),
  generate_response("สัญลักษ์ของ คณะ คืออะไร ?"),
  generate_response("หลักสูตรปริญญาตรี มีอะไรบ้าง ?"),
  generate_response("หลักสูตรปริญญาโท มีอะไรบ้าง ?"),
  generate_response("หลักสูตรปริญญาเอก มีอะไรบ้าง ?"),
  generate_response("ความเป็นมาของคณะ มีอะไรบ้าง ?"),
]
        
for r in result:
    print(r)
    print("==="*20)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

คณบดีคือใคร ? : คณบดีคือ ผศ. ดร.ณรงค์ฤทธิ์ วราภรณ์
สัญลักษ์ของ คณะ คืออะไร ? : สัญลักษณ์ประจำคณะเทคโนโลยีสารสนเทศของมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรีคือ “Information Evolution” ซึ่งเป็นประติมากรรมที่ออกแบบและสร้างโดยอาจารย์ชำเรือง วิเชียรเขตต์ ศิลปินแห่งชาติ สาขาทัศนศิลป์ (ประติมากรรม)
หลักสูตรปริญญาตรี มีอะไรบ้าง ? : ข้อมูลที่มีเกี่ยวกับหลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) และหลักสูตรวิทยาศาสตรมหาบัณฑิต สาขาวิชาวิทยาการคอมพิวเตอร์ คือ:

- หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาระบบสารสนเทศอัจฉริยะทางธุรกิจ 
- หลักสูตรวิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิทยาการคอมพิวเตอร์
หลักสูตรปริญญาโท มีอะไรบ้าง ? : มีหลักสูตรดังต่อไปนี้

1. วิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาวิทยาการคอมพิวเตอร์
2. วิทยาศาสตรมหาบัณฑิต (วท.ม.) สาขาวิชาเทคโนโลยีสารสนเทศ
หลักสูตรปริญญาเอก มีอะไรบ้าง ? : ข้อมูลที่มีอยู่ไม่มีข้อมูลเกี่ยวกับหลักสูตรปริญญาเอก (Ph.D.) ของมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี คณะเทคโนโลยีสารสนเทศ
ความเป็นมาของคณะ มีอะไรบ้าง ? : ข้อมูลที่มีอยู่เกี่ยวกับความเป็นมาของคณะเทคโนโลยีสารส